# Лабораторная работа №1
## Выполнил студент группы БФИ2302 АндреевАМ

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)
4. [Вывод](#Вывод)

### Задание №1
<i> Вызвать функцию print() и передать туда строку Hello, World! </i>

In [ ]:
print("Hello, World!")

### Задание №2
Написать генератор случайных матриц(многомерных), который принимает
опциональные параметры <b>m</b>, <b>n</b>, <b>min_limit</b>, <b>max_limit</b>, где <b>m</b> и <b>n</b> указывают размер
матрицы, а <b>min_lim</b> и <b>max_lim</b> - минимальное и максимальное значение для
генерируемого числа.

In [ ]:
import random
import time
import numpy as np

In [ ]:
def userInput(prompt, default):
    u = input(prompt)
    return int(u) if u.strip() else default

def argProvider (func):
    def wrapper():
        m = userInput("Rows: ",50)
        n = userInput("Columns: ",50)
        user_min = userInput("MinRand: ",-250)
        user_max = userInput("MaxRand: ",1000)
        value = func(m,n,user_min,user_max)
        return value
    return wrapper

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        value = func(*args, **kwargs)
        print("--- {0} ms ---".format(round((time.time() - start_time)*1000)))
        return value
    return wrapper

@argProvider
@timer
def nestedFor(m,n,user_min,user_max):
    matrix = []
    for row in range(m):
        a = []
        for col in range(n):
            a.append(random.randint(user_min,user_max))
        matrix.append(a)
    return matrix

@argProvider
@timer
def listComp(m,n,user_min,user_max):
    return [[random.randint(user_min,user_max) for _ in range(n)] for _ in range(m)]

@argProvider
@timer
def numPy(m,n,user_min,user_max):
    return np.random.randint(user_min,user_max+1,size=(m,n))

In [ ]:
generated = nestedFor()
for row in generated:
    print(row)

In [ ]:
generated = listComp()
for row in generated:
    print(row)

In [ ]:
generated = numPy()
for row in generated:
    print(row)

### Задание №3
Реализовать методы сортировки строк числовой матрицы в соответствии с
заданием. Оценить время работы каждого алгоритма сортировки и сравнить его со
временем стандартной функции сортировки. Испытания проводить на сгенерированных
матрицах.

In [ ]:
# calls generator, prints generated list, passes it to sorter, prints sorted list
def driver(func):
    def wrapper():
        matrix_generated = listComp()
        for row in matrix_generated:
            print(row)
        matrix_sorted = func(matrix_generated)
        for row in matrix_sorted:
            print(row)
    return wrapper

# transforms 2D list to 1D and back
def flatten(func):
    def wrapper(matrix):
        flat_matrix = [elem for row in matrix for elem in row]
        flat_matrix = func(flat_matrix)
        sorted_matrix = []
        m = len(matrix)
        n = len(matrix[0])
        for i in range(m):
            start = i * n
            end = start + n
            sorted_matrix.append(flat_matrix[start:end])
        return sorted_matrix
    return wrapper

In [ ]:
# Сортировка выбором.
# Берётся срез массива, в котором минимальный элемент переносят в самый левый угол,
# после чего срез уменьшается и цикл повторяется.

# sorting 2D list directly
@driver
@timer
def bruteForce(matrix):
    print(matrix)
    m = len(matrix)
    n = len(matrix[0])
    for row in range(m):
        min_row=row
        for col in range(n):
            min_col=col
            for next_row in range(row,m):
                unsorted = range(col,n) if next_row==row else range(n)
                for next_col in unsorted:
                    if matrix[next_row][next_col]<=matrix[min_row][min_col]:
                        min_row=next_row
                        min_col=next_col
            matrix[row][col],matrix[min_row][min_col]=matrix[min_row][min_col],matrix[row][col]
    return matrix

# sorting transformed 1D list
@driver
@flatten
@timer
def selectSort(matrix):
    n = len(matrix)
    for i in range(n):             
        min_idx = i
        for j in range(i+1, n):
            if matrix[j] < matrix[min_idx]:
                min_idx = j
        matrix[i], matrix[min_idx] = matrix[min_idx], matrix[i]
    return matrix


In [ ]:
bruteForce()

In [ ]:
selectSort()

In [ ]:
# Сортировка вставкой.
@driver
@flatten
@timer
def insertionSort(matrix):
    for i in range(1, len(matrix)):
        buffer = matrix[i]
        j = i - 1
        while j >= 0 and matrix[j] > buffer:
            matrix[j+1] = matrix[j]
            j -= 1
        matrix[j+1] = buffer
    return matrix

In [ ]:
insertionSort()

In [ ]:
# Сортировка обменом. || Сортировка пузырьком.

@driver
@flatten
@timer
def bubbleSort(matrix):
    n = len(matrix)
    for i in range(n):
        swapped = False
        for j in range(0, n-i-1):
            if matrix[j] > matrix[j+1]:
                matrix[j],matrix[j+1] = matrix[j+1],matrix[j]
                swapped = True
        if swapped == False:
            break
    return matrix

In [ ]:
bubbleSort()

In [ ]:
# Сортировка Шелла.

@driver
@flatten
@timer
def shellSort(matrix):
    n = len(matrix)
    gap = n // 2
    while gap > 0:
        for i in range(gap, n):
            buffer = matrix[i]
            j = i
            while j >= gap and matrix[j - gap] > buffer:
                matrix[j] = matrix[j - gap]
                j -= gap
            matrix[j] = buffer
        gap //= 2
    return matrix

In [ ]:
shellSort()

In [ ]:
# Быстрая сортировка.

@driver
@flatten
@timer
def quickSort(matrix):
    def swap(matrix, i, j):
            matrix[i], matrix[j] = matrix[j], matrix[i]
    def partition(matrix, low, high):
        pivot = matrix[high]
        i = low - 1
        for j in range(low, high):
            if matrix[j] < pivot:
                i += 1
                swap(matrix, i, j)
        swap(matrix, i + 1, high)
        return i+1
    def recursion(matrix, low, high):
        if low < high:
            pi = partition(matrix, low, high)
            recursion(matrix, low, pi-1)
            recursion(matrix, pi+1, high)
    n = len(matrix)
    recursion(matrix, 0, n-1)
    return matrix

In [ ]:
quickSort()

In [22]:
# Турнирная сортировка.

@driver
@flatten
@timer
def heapSort(matrix):
    def heapify(matrix, n, i):
        largest = i 
        l = 2 * i + 1 
        r = 2 * i + 2  
        if l < n and matrix[l] > matrix[largest]:
            largest = l
        if r < n and matrix[r] > matrix[largest]:
            largest = r
        if largest != i:
            matrix[i], matrix[largest] = matrix[largest], matrix[i]  # Swap
            heapify(matrix, n, largest)
    n = len(matrix) 
    for i in range(n // 2 - 1, -1, -1):
        heapify(matrix, n, i)
    for i in range(n - 1, 0, -1):
        matrix[0], matrix[i] = matrix[i], matrix[0] 
        heapify(matrix, i, 0)
    return matrix


In [ ]:
heapSort()

### Вывод
